In [ ]:
import pandas as pd
df2_train = pd.read_csv("training_task_b_source_tweets.csv")
df2_test = pd.read_csv("test_task_b_tweets.csv")

In [ ]:
df2_train.head()

,id,in_reply_to_status_id_str,text,user.verified,user.followers_count,retweet_count,favorite_count,entities.hashtags,entities.urls,label
0,552783667052167168,NaN,France: 10 people dead after shooting at HQ of...,True,129573,486,38,"[{'indices': [74, 87], 'text': 'CharlieHebdo'}]",[],true
1,552785375161499649,NaN,BREAKING: 10 reportedly shot dead at Paris HQ ...,True,972167,128,5,[],"[{'url': 'http://t.co/5F1DOwzoCQ', 'indices': ...",true
2,552791196247269378,NaN,BREAKING: At least 10 killed in shooting at Fr...,True,3029912,295,78,[],"[{'url': 'http://t.co/mnAeA7j7fY', 'indices': ...",true
3,552791578893619200,NaN,Eleven dead in shooting at Paris offices of sa...,True,3091451,338,28,[],"[{'url': 'http://t.co/gOtSqbwROc', 'indices': ...",true
4,552792544132997121,NaN,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,True,973212,203,32,[],"[{'url': 'http://t.co/kjW3uwCV0f', 'indices': ...",true


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing

In [ ]:
import nltk
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df2_train.label.unique()

array(['true', 'false', 'unverified'], dtype=object)

In [ ]:
df2_train["text"]=df2_train["text"].str.lower()
df2_test["text"]=df2_test["text"].str.lower()

In [ ]:
def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)
def stop_words(text):
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  return text
def getStemmText(text):
 tokens=word_tokenize(text)
 ps = PorterStemmer()
 tokens=[ps.stem(word) for word in tokens]
 return ' '.join(tokens)

In [ ]:
df2_train['text'] = list(map(stop_words,df2_train['text']))
df2_train['text'] = list(map(getLemmText,df2_train['text']))
df2_train['text'] = list(map(getStemmText,df2_train['text']))

df2_test['text'] = list(map(stop_words,df2_test['text']))
df2_test['text'] = list(map(getLemmText,df2_test['text']))
df2_test['text'] = list(map(getStemmText,df2_test['text']))

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df2_train['label'] = label_encoder.fit_transform(df2_train.label.values)
df2_test['label'] = label_encoder.fit_transform(df2_test.label.values)

In [ ]:
print(df2_test['label'].value_counts())

0    30
1    22
2     4
Name: label, dtype: int64


In [ ]:
df2_train.drop(df2_train.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
df2_test.drop(df2_test.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)

In [ ]:
df2_test.head()

,text,label
0,# break pentagon releas video mother bomb drop...,0
1,rep sheila jackson lee shame still laugh insis...,1
2,icymi rep sheila jackson lee dtx want hurrican...,1
3,2003 houston u rep sheila jackson lee began ca...,1
4,noah ark theme park destroy flood http tco yjp...,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
stopset = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

y_train = df2_train["label"]
y_test = df2_test["label"]
x_train = vectorizer.fit_transform(df2_train["text"])
x_test = vectorizer.transform(df2_test["text"])

In [ ]:
classifier = MultinomialNB().fit(x_train, y_train)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
import sklearn.metrics as metrics 
print(metrics.confusion_matrix(y_test,y_pred),"\n")
print(metrics.accuracy_score(y_test,y_pred))
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="macro"))

[[ 1 27  2]
 [ 0 21  1]
 [ 0  3  1]] 

0.4107142857142857
0.5539215686274509
0.4126262626262626
0.2966195315952276


In [ ]:
y_prob = classifier.predict_proba(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
import math
y_true = [np.argmax(val) for val in y_test]
conf = []
for i in range(len(y_test)):
  conf.append(max(y_prob[i]))
RMSE = mean_squared_error([1]*len(conf), conf,squared = False)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.4983649761574323


In [ ]:
# using support vector machines for classification
from sklearn import svm
from sklearn.metrics import classification_report

classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(x_train, y_train)
prediction_linear = classifier_linear.predict(x_test)

In [ ]:
print(metrics.confusion_matrix(y_test,prediction_linear),"\n")
print(metrics.accuracy_score(y_test,prediction_linear))

print(precision_score(y_test, prediction_linear , average="macro"))
print(recall_score(y_test, prediction_linear , average="macro"))
print(f1_score(y_test, prediction_linear , average="macro"))

[[ 3 20  7]
 [ 0 16  6]
 [ 0  3  1]] 

0.35714285714285715
0.49389499389499386
0.35909090909090907
0.27250648562123975


In [ ]:
from sklearn.metrics import mean_squared_error
import math
y_true = [np.argmax(val) for val in y_test]
conf = []
for i in range(len(y_test)):
  conf.append(max(y_prob[i]))
RMSE = mean_squared_error([1]*len(conf), conf,squared = False)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.4983649761574323
